#Use a NN to model XOR

Study how individual motifs work to compose into larger circuits. Just like how studying AND gate composes into larger computer circuit, or how studying cells and particles enables understanding on a macroscopic level.

Not for a research paper, but for personal understanding, which will help for future research papers.

XOR is a geometric motif in latent space. Do other such motifs exist? Perhaps there is an analogous mathematical calculation to describe these motifs, and that generalizes these observations of this notebook.

In [1]:
%%capture
import torch
import copy
import matplotlib.pyplot as plt
import numpy as np
import os 

In [2]:
class Feedforward(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Feedforward, self).__init__()
        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)  # one hidden layer
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.hidden_size, 1)  #output layer w/o sigmoid
        self.sigmoid = torch.nn.Sigmoid()
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        output = self.sigmoid(output)
        return output

In [3]:
model = Feedforward(2, 2)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [4]:
x_train = torch.FloatTensor(np.array([[0,0], [0,1], [1,0], [1,1]]))
y_train = torch.FloatTensor(np.array([0,1,1,0]))

The model training is different every time, so keep training from scratch until get one with low loss. Sometimes it may get stuck, sometimes it goes very low. This is due to initial conditions interacting with GD, as gradient descent is not random.

In [ ]:
model.train()
epoch = 15000

for epoch in range(epoch):
    #sets the gradients to zero before we start backpropagation. 
    #This is a necessary step as PyTorch accumulates the gradients from the backward passes from the previous epochs.
    optimizer.zero_grad()
    # Forward pass
    y_pred = model(x_train)
    # Compute Loss
    loss = criterion(y_pred.squeeze(), y_train)
   
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()

In [7]:
model.eval()
y_pred = model(x_train)
y_pred

tensor([[0.0230],
        [0.9941],
        [0.9941],
        [0.0230]], grad_fn=<SigmoidBackward0>)

In [6]:
PATH = "state_dict_model.pt"
torch.save(model.state_dict(), PATH)



---
# Get activations


In [8]:
def get_activations(input, layer_name, model):
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    for name_to_check, layer in model.named_modules():
        if name_to_check == layer_name:
            break
    layer.register_forward_hook(get_activation(layer_name))
    
    output = model(input)

    return activation.copy()  #else will return the same actvs of model

In [9]:
# get previous last layer name
named_layers = dict(model.named_modules())
layers = list(named_layers.keys())

# too many branches, so just get the converged branch points
# '' is the entire model, so disregard it
# layers = [x for x in layers if '.' not in x and x != '']  
layers = [x for x in layers if x != '']  
layers

['fc1', 'relu', 'fc2', 'sigmoid']



---
# Compare different input activations

Compare [0, 0] and [1, 0]

In [10]:
x_train[0]

tensor([0., 0.])

In [11]:
input_1 = x_train[0]
out = model(input_1)
print(out)

tensor([0.0230], grad_fn=<SigmoidBackward0>)


In [12]:
for layer_name in layers:
    print(get_activations(input_1, layer_name, model))

{'fc1': tensor([-1.8150e-05, -1.6780e-04])}
{'relu': tensor([0., 0.])}
{'fc2': tensor([-3.7482])}
{'sigmoid': tensor([0.0230])}


In [13]:
input_2 = x_train[0] + torch.tensor([1,0])
input_2

tensor([1., 0.])

In [14]:
for layer_name in layers:
    print(get_activations(input_2, layer_name, model))

{'fc1': tensor([-2.5164,  2.4917])}
{'relu': tensor([0.0000, 2.4917])}
{'fc2': tensor([5.1242])}
{'sigmoid': tensor([0.9941])}


In [15]:
for layer_name in layers:
    print( '[0,0]:', get_activations(input_1, layer_name, model), '[1,0]:', get_activations(input_2, layer_name, model))

[0,0]: {'fc1': tensor([-1.8150e-05, -1.6780e-04])} [1,0]: {'fc1': tensor([-2.5164,  2.4917])}
[0,0]: {'relu': tensor([0., 0.])} [1,0]: {'relu': tensor([0.0000, 2.4917])}
[0,0]: {'fc2': tensor([-3.7482])} [1,0]: {'fc2': tensor([5.1242])}
[0,0]: {'sigmoid': tensor([0.0230])} [1,0]: {'sigmoid': tensor([0.9941])}


In [16]:
for layer_name in layers:
    diff = get_activations(input_2, layer_name, model)[layer_name] - get_activations(input_1, layer_name, model)[layer_name]
    print(diff)

tensor([-2.5164,  2.4918])
tensor([0.0000, 2.4917])
tensor([8.8724])
tensor([0.9711])




---

# Why does this input change cause this difference?

Why are the activations different? Let's look at how each activation is calculated, and how the change in input causes the change in activation.

In [17]:
model

Feedforward(
  (fc1): Linear(in_features=2, out_features=2, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=2, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [18]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-2.5164,  2.5164],
        [ 2.4918, -2.4921]], requires_grad=True)
Parameter containing:
tensor([-1.8150e-05, -1.6780e-04], requires_grad=True)
Parameter containing:
tensor([[3.5254, 3.5608]], requires_grad=True)
Parameter containing:
tensor([-3.7482], requires_grad=True)


In [19]:
prev_weights = model.fc1.weight.data
prev_weights

tensor([[-2.5164,  2.5164],
        [ 2.4918, -2.4921]])

In [20]:
for layer in layers:
    if isinstance(named_layers[layer], torch.nn.Linear):
        print(layer, named_layers[layer].state_dict()['bias'])

fc1 tensor([-1.8150e-05, -1.6780e-04])
fc2 tensor([-3.7482])


In [33]:
torch.matmul(model.fc1.weight.data, input_1) + named_layers['fc1'].state_dict()['bias']

tensor([-1.8150e-05, -1.6780e-04])

In [35]:
print(get_activations(input_1, 'fc1', model))

{'fc1': tensor([-1.8150e-05, -1.6780e-04])}


In [34]:
torch.matmul(model.fc1.weight.data, input_2)  + named_layers['fc1'].state_dict()['bias']

tensor([-2.5164,  2.4917])



---

Break down each step of W * X + b

In [36]:
model.fc1.weight.data

tensor([[-2.5164,  2.5164],
        [ 2.4918, -2.4921]])

In [37]:
input_2

tensor([1., 0.])

Get:

[$w_{11} * x_1$ $w_{12} * x_2$]

[$w_{21} * x_1$ $w_{22} * x_2$]

In [38]:
WX_hadamard = torch.multiply(model.fc1.weight.data, input_2)
WX_hadamard

tensor([[-2.5164,  0.0000],
        [ 2.4918, -0.0000]])

WX:

[$w_{11}$ $w_{12}$]

[$w_{21}$ $w_{22}$]

w11 * (1) - w12 * 0 = w11

w21 * (1) + w22 * 0 = w21

Thus, when you change the input from [0, 0] to [1, 0], you are changing WX from [0, 0] to [w11, w21]. And we know that [1, 0] is supposed to be 1.

Likewise, [0, 1] as input gets:

[$w_{11} * x_1$ $w_{12} * x_2$] = [$w_{12}$]

[$w_{21} * x_1$ $w_{22} * x_2$] = [$w_{22}$]

But we also need [1, 1] to be 0, so it's not as simple as "make it so the neuron outputs are bigger". 

In [39]:
input_3 = torch.FloatTensor(np.array([1,1]))
input_3

tensor([1., 1.])

In [41]:
for layer_name in layers:
    print(get_activations(input_3, layer_name, model))

{'fc1': tensor([-2.6733e-05, -3.7474e-04])}
{'relu': tensor([0., 0.])}
{'fc2': tensor([-3.7482])}
{'sigmoid': tensor([0.0230])}


In [43]:
for layer_name in layers:
    diff = get_activations(input_3, layer_name, model)[layer_name] - get_activations(input_1, layer_name, model)[layer_name]
    print(diff)

tensor([-8.5831e-06, -2.0695e-04])
tensor([0., 0.])
tensor([0.])
tensor([0.])


[0, 0] and [1, 1] have the SAME neuron outputs after fc1. Let's look at the weights of fc1 again to see why that's the case.

In [44]:
model.fc1.weight.data

tensor([[-2.5164,  2.5164],
        [ 2.4918, -2.4921]])

In [45]:
WX_hadamard = torch.multiply(model.fc1.weight.data, input_3)
WX_hadamard

tensor([[-2.5164,  2.5164],
        [ 2.4918, -2.4921]])

The first element of WX is 0 or close to 0.

The second element is also close to 0.

So the weights were learned to make sure w11 - w12 was close to 0, as ReLU would turn that into 0.

Then in fc2 (output node), the bias is negative so that sigmoid makes it less than 0. 

But for [0,1] and [1,0], w11 - w12 is nonzero. The more different they are, the more one will be expressed than the other. But when x1 and x2 are close, because w11 and w12 are also close, the first output of fc1 (w11*x1 + w12*x2) is close to 0.







# Why hidden layer for two elements?

Matrix multiplication is unit conversion. Here, the first element of WX for fc1 is measuring how close x1 and x2 are. But the second element is doing the same thing. Is it even necessary?

Yes, it's necessary for [0, 1] and [1, 0]. Notice **because w11 or w12 must be negative, and the other must be positive** (WLOG say w11 is positive), then [1, 0] will return w11, a positive (a non-zero after ReLU) for the first element, but [0, 1] will return w12, a negative (a zero after ReLU). 

Even without ReLU, we want to output POSITIVE 1, not negative 1. So this cannot work.

Thus, we need the second row (second element of fc1) so that [0, 1] can return a positive (a non-zero after ReLU).



---
# Gradual input modification 

Now slightly modify the input and see what happens to each layer!

Try different modification levels. As you gradually change it, how do the weights change?


---


Use:
torch.tensor([0.0, 0.0]) instead of torch.tensor([0, 0])

https://stackoverflow.com/questions/60440292/runtimeerror-expected-scalar-type-long-but-found-float

In [52]:
layer_name = 'fc1'
# step_incr = [round(x * 0.1, 1) for x in range(0, 10)]
old_input = torch.tensor([0.0, 0.0])
new_input = torch.tensor([0.0, 0.0])
# for i in step_incr:
for i in range(0, 10):
    old_input = new_input.clone()
    new_input = old_input + torch.tensor([0.1, 0])
    # new_input = torch.tensor([i, 0])
    print(round(0.1 * i, 1))
    diff = get_activations(new_input, layer_name, model)[layer_name] - get_activations(old_input, layer_name, model)[layer_name]
    print(diff)

0.0
tensor([-0.2516,  0.2492])
0.1
tensor([-0.2516,  0.2492])
0.2
tensor([-0.2516,  0.2492])
0.3
tensor([-0.2516,  0.2492])
0.4
tensor([-0.2516,  0.2492])
0.5
tensor([-0.2516,  0.2492])
0.6
tensor([-0.2516,  0.2492])
0.7
tensor([-0.2516,  0.2492])
0.8
tensor([-0.2516,  0.2492])
0.9
tensor([-0.2516,  0.2492])


It's the same increment every time, since WX+b is linear.

Now instead of comparing with previous, compare it with original, [0,0]

In [61]:
layer_name = 'fc1'
# old_input = torch.tensor([0.0, 0.0])
new_input = torch.tensor([0.0, 0.0])
print('orig actvs: ', get_activations(torch.tensor([0.0, 0.0]), layer_name, model)[layer_name] )
for i in range(1, 10):
    old_input = new_input.clone()
    new_input = old_input + torch.tensor([0.1, 0])
    print([round(0.1 * i, 1), 0])
    actv_new = get_activations(new_input, layer_name, model)[layer_name]
    # actv_orig = get_activations(torch.tensor([0.0, 0.0]), layer_name, model)[layer_name]
    # diff = actv_new - actv_orig
    # print(torch.round(diff, decimals=1))
    print(actv_new)

orig actvs:  tensor([-1.8150e-05, -1.6780e-04])
[0.1, 0]
tensor([-0.2517,  0.2490])
[0.2, 0]
tensor([-0.5033,  0.4982])
[0.3, 0]
tensor([-0.7549,  0.7474])
[0.4, 0]
tensor([-1.0066,  0.9966])
[0.5, 0]
tensor([-1.2582,  1.2458])
[0.6, 0]
tensor([-1.5099,  1.4949])
[0.7, 0]
tensor([-1.7615,  1.7441])
[0.8, 0]
tensor([-2.0131,  1.9933])
[0.9, 0]
tensor([-2.2648,  2.2425])


Review the actual activation diffs b/w [0, 0] and [1, 0]

In [64]:
for layer_name in layers:
    print( '[0,0]:', get_activations(input_1, layer_name, model), '[1,0]:', get_activations(input_2, layer_name, model))

[0,0]: {'fc1': tensor([-1.8150e-05, -1.6780e-04])} [1,0]: {'fc1': tensor([-2.5164,  2.4917])}
[0,0]: {'relu': tensor([0., 0.])} [1,0]: {'relu': tensor([0.0000, 2.4917])}
[0,0]: {'fc2': tensor([-3.7482])} [1,0]: {'fc2': tensor([5.1242])}
[0,0]: {'sigmoid': tensor([0.0230])} [1,0]: {'sigmoid': tensor([0.9941])}


Before, we saw that going from [0, 0] to [1, 0] makes WX go from around [0, 0] to [w11, w21]. For fc1_1, we see a decrease from ~0 to w11 as x1 gets larger. Similarly, fc1_2 is closer to w21 as x gets larger. 

Clearly due to ReLU, w11 is not needed in [x1, 0] (st x1 > 0) because w11 is negative and thus will always be turned to 0. It only cares about increasing fc1_2 from 0 to w21. But for [0, 1], [fc1_1, fc1_2] = [w12, w22] where w12 > 0 and w22 < 0, so in that case, the first element fc1_1 increases. The [0,1] and [1,0] have to ensure that at least one output of fc1's (WX + b) is positive.

If [fc1_1 and fc1_2] are close to 0, since the bias is negative, the relu will make them [0,0]. 

(Keep in mind this model was only overfitted on 4 data pts, so it doesn't do well for data pts in b/w.)



---

Let's gradually change along the diagonal from [0,0] to [1,1]

In [63]:
layer_name = 'fc1'
old_input = torch.tensor([0.0, 0.0])
new_input = torch.tensor([0.0, 0.0])
for i in range(1, 10):
    old_input = new_input.clone()
    new_input = old_input + torch.tensor([0.1, 0.1])
    print([round(0.1 * i, 1), round(0.1 * i, 1)])
    actv_new = get_activations(new_input, layer_name, model)[layer_name]
    actv_orig = get_activations(torch.tensor([0.0, 0.0]), layer_name, model)[layer_name]
    diff = actv_new - actv_orig
    print(diff)

[0.1, 0.1]
tensor([-8.3447e-07, -2.0698e-05])
[0.2, 0.2]
tensor([-1.6689e-06, -4.1395e-05])
[0.3, 0.3]
tensor([-2.5630e-06, -6.2108e-05])
[0.4, 0.4]
tensor([-3.3379e-06, -8.2791e-05])
[0.5, 0.5]
tensor([-4.2915e-06, -1.0347e-04])
[0.6, 0.6]
tensor([-5.1260e-06, -1.2422e-04])
[0.7, 0.7]
tensor([-6.0797e-06, -1.4484e-04])
[0.8, 0.8]
tensor([-6.9141e-06, -1.6558e-04])
[0.9, 0.9]
tensor([-7.6294e-06, -1.8620e-04])


Along this diagonal, [fc1_1 and fc1_2] always stay negative and close to 0, unlike going from [0, 0] to [1, 0], where fc1_1 decreases (getting closer to w11, which is -2.5) and fc1_2 increases (getting closer to w12, which is 2.5).

This is because fc1_1 = w11 + w12, and w11 ~= -w12, so whenever the coefficients x1 and x2 are close in value, so will x1 * w11 and x2 * w12. 



---

Conditions to allow NN to model XOR: (one possible state)

- fc1_1: the weights were learned to make sure w1 - w2 was close to 0 (ie. w11 ~= -w12)
- fc1_2: if w11 is negative, w21 should be positive, and vv. if w12 is negative, w22 should be positive, and vv.
- fc2's bias should be negative to ensure WX after fc2, if 0, will make it neg, thus making sigmoid make WX+b be close to 0

Any NN with the architecture above whose weights satisfy these conditions will model XOR.

Thus, we can possibly analogously match NNs satisfying these same conditions by their motifs. 

Put in these constraints then use a constraint solver to find the weights needed.

**Additionally**, we may be able to find 'training patterns' which allow the NN training algorithms to figure out which motifs it needs in a 'meaningful' way, matching regions to 'meaning' that corresponds to the data, instead of simply using gradient descent and other optimization methods. We combine 'training patterns' with other methods such as GD and neuro-evolution.



---

Change of basis, human-like interpretation (approx.):

For fc1, the first row of WX measures how similar $x_1$ and $x_2$ are. The second row does the same, but is neg when the first row is pos, and vv.



---

Now these NN motifs are how weights are related to one another. For instance, w11 ~= -w12. Any sub-network satisfying these constraints is "that motif". Finding these motifs across a large NN means finding patterns that satisfy each condition. Each condition is stricter, making a more specific and larger pattern query, so adding more conditions means less matches.

These are not 'exact' patterns, but analogous, since weights are RELATIVE to one another. For instance, w11 > w12, not requiring w11 to exactly have some value (though it may).

Note that even the network topology may not be the same. You can have a path, such that it's not that one neuron that's a neighbor needs to be related to another in such a way, but that another neuron somewhere down the path has that relation.

Finding these conditions does not need human interpretation- these general motif conditions may be inferred by an algorithm (learned or not) by taking in multiple NNs with the same role. This 'meta learning' will take in NNs as training data and find the common patterns they all have.

This meta NN will be applied to networks. Then you can apply it to a NN to find subgraph patterns that it says 'yes' on. It scans the larger NN to find potential areas, growing larger from those sub-areas (this may be a DRL problem). 



---

# Try training it again and look at the weights

For multiple training instances, does it do this every time? There may be multiple ways for an ANN circuit to calculate XOR.

In [70]:
model_2 = Feedforward(2, 2)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model_2.parameters(), lr = 0.01)

# if you continue training, usually will get stuck?

x_train = torch.FloatTensor(np.array([[0,0], [0,1], [1,0], [1,1]]))
y_train = torch.FloatTensor(np.array([0,1,1,0]))

In [ ]:
model_2.train()
epoch = 15000

for epoch in range(epoch):
    #sets the gradients to zero before we start backpropagation. 
    #This is a necessary step as PyTorch accumulates the gradients from the backward passes from the previous epochs.
    optimizer.zero_grad()
    # Forward pass
    y_pred = model_2(x_train)
    # Compute Loss
    loss = criterion(y_pred.squeeze(), y_train)
   
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()

In [72]:
model_2.eval()
y_pred = model_2(x_train)
y_pred

tensor([[0.0125],
        [0.9578],
        [0.9578],
        [0.0156]], grad_fn=<SigmoidBackward0>)

In [74]:
PATH = "state_dict_model_2.pt"
torch.save(model_2.state_dict(), PATH)

In [80]:
for param in model_2.parameters():
    print(param)

Parameter containing:
tensor([[ 2.0555,  2.0556],
        [-2.1004, -2.1004]], requires_grad=True)
Parameter containing:
tensor([-2.0558,  2.1001], requires_grad=True)
Parameter containing:
tensor([[-3.5337, -3.5672]], requires_grad=True)
Parameter containing:
tensor([3.1211], requires_grad=True)


This time is strange. Now, w11 = w12, and w21 = w22. But w11 and w12 are positive, while w21 and w22 are negative. So this is a second set of conditions?


In [75]:
for layer_name in layers:
    print(get_activations(input_1, layer_name, model_2))

{'fc1': tensor([-2.0558,  2.1001])}
{'relu': tensor([0.0000, 2.1001])}
{'fc2': tensor([-4.3703])}
{'sigmoid': tensor([0.0125])}


Notice the activations after fc1 are just the bias.

In [78]:
for layer_name in layers:
    print(get_activations(input_2, layer_name, model_2))

{'fc1': tensor([-0.0003, -0.0003])}
{'relu': tensor([0., 0.])}
{'fc2': tensor([3.1211])}
{'sigmoid': tensor([0.9578])}


In [77]:
for layer_name in layers:
    print(get_activations(input_3, layer_name, model_2))

{'fc1': tensor([ 2.0553, -2.1007])}
{'relu': tensor([2.0553, 0.0000])}
{'fc2': tensor([-4.1417])}
{'sigmoid': tensor([0.0156])}


Compared to [0, 0], the fc activations have opp signs. 

In [82]:
input_4 = torch.FloatTensor(np.array([0,1]))
for layer_name in layers:
    print(get_activations(input_4, layer_name, model_2))

{'fc1': tensor([-0.0002, -0.0003])}
{'relu': tensor([0., 0.])}
{'fc2': tensor([3.1211])}
{'sigmoid': tensor([0.9578])}


We see for both [0, 1] and [1, 0], now the key is that the fc1 are close to 0. This pattern means the second class, regardless of being 0 or 1, is 'zeroed out'. Essentially, a reset, to allow second layer to do something new. Once 'zeroed out', the bias 3.1211 is able to be added. It's positive, meaning sigmoid will turn it close to 1.

Now, the reason why [0, 0] and [1, 1] differ from the 'opposing x1 and x2' is because after the (first and only) ReLU, there is one element that is positive. Since the fc2 weights are both negative and sufficiently big enough, this means fc2 will ALWAYS give a negative because w*x means negative * positive = negative, and they're big enough to always ensure adding the bias won't make wx+b be >0. In contrast, the [1, 0] and [0, 1] would 'zero out' and ignore fc2's weights, depending only on the positive bias.

These are strategies that can be used throughout the NN. They are not packaged into 'human concept' units like 'cat eye' or 'cat paw', but they are still patterns that one can interpret in the ANN circuit. Perhaps these patterns exist throughout ANNs.

---
# Manually modify weights to try to create XOR, based on conditions

We can try manually inserting weights that meet those conditions generalized from model 1 (based on reasons why model 1 works).

In [4]:
model_3 = Feedforward(2, 2)

In [9]:
model_3.fc1.weight.data = torch.FloatTensor(np.array([[1, -1], [-1,1]]))

In [10]:
model_3.fc1.weight.data

tensor([[ 1., -1.],
        [-1.,  1.]])

In [16]:
model_3.fc2.bias.data = torch.FloatTensor(np.array([-1]))

In [18]:
model_3.eval()
y_pred = model_3(x_train)
y_pred

tensor([[0.3509],
        [0.2839],
        [0.5076],
        [0.3509]], grad_fn=<SigmoidBackward0>)

That's not accurate. Analyze what happened in the activations.

In [ ]:
for layer_name in layers:
    print(get_activations(input_1, layer_name))



---

# Try using single layer perceptron

Matrix multiplication is unit conversion. Here, the first element of WX for fc1 is measuring how close x1 and x2 are. But the second element is doing the same thing. Is it even necessary?